Required initial steps:
    
    * The raw text file from https://afltables.com/afl/stats/biglists/bg3.txt is saved as llist.txt
    * The file is then processed through the bash script file a_updater.sh to convert it to a csv and make other modifications, by typing ./a_updater into Terminal.
    * The bash file saves the processed file as newlist.csv, ready to be imported from within Python

# Round 23 regular season completion 2018

In [ ]:
#%%html
#<img src="img/ess_stk.png" alt="Drawing" align="left" style="width: 800px;"/>

In [1]:
import pandas as pd
cols = ['seq','date','round','team1','gls_1','bhs_1','pts_1','team2','gls_2','bhs_2','pts_2','venue']

In [2]:
upd_round = pd.read_csv("/Users/stevegabriel/data_projects/sports_data/newlist.csv", names = cols)
trans = upd_round

In [3]:
# Modify index to start at 1 
trans.index += 1
trans.tail()

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,pts_2,venue
15394,15394,25-Aug-2018,R23,Carlton,8,13,61,Adelaide,26,9,165,Docklands
15395,15395,25-Aug-2018,R23,Sydney,10,14,74,Hawthorn,12,11,83,S.C.G.
15396,15396,26-Aug-2018,R23,Brisbane Lions,11,6,72,West Coast,14,14,98,Gabba
15397,15397,26-Aug-2018,R23,Melbourne,15,12,102,GW Sydney,8,9,57,M.C.G.
15398,15398,26-Aug-2018,R23,St Kilda,14,10,94,North Melbourne,17,15,117,Docklands


In [4]:
# Add states
nsw = ['S.C.G.','Sydney Showground','Stadium Australia','Blacktown','Albury']
qld = ['Gabba','Carrara','Cazaly\'s Stadium','Brisbane Exhibition']
sa = ['Football Park','Adelaide Oval']
wa = ['W.A.C.A.','Perth Stadium','Subiaco']
nt = ['Marrara Oval','Traeger Park']
tas = ['York Park','Bellerive Oval','North Hobart']
act = ['Manuka Oval','Bruce Stadium']
intl = ['Wellington','Jiangwan Stadium']

In [5]:
def locations(x):
    if x in nsw:
        return "NSW"
    elif x in sa:
        return "SA"
    elif x in wa:
        return "WA"
    elif x in nt:
        return "NT"
    elif x in tas:
        return "TAS"
    elif x in intl:
        return "INTL"
    elif x in act:
        return "ACT"
    elif x in qld:
        return "QLD"
    else: 
        return "VIC"

In [6]:
trans["state"] = trans["venue"].apply(locations)
trans["state"].value_counts()

VIC     13032
WA        640
SA        573
NSW       511
QLD       474
TAS        91
ACT        48
NT         24
INTL        5
Name: state, dtype: int64

In [7]:
## 1. HVAR & AGGR
trans['hvar'] = trans['pts_1'] - trans['pts_2']
trans['aggr'] = trans['pts_1'] + trans['pts_2']

In [8]:
## 2. YEAR & MONTH
import time
import datetime

In [9]:
trans['date'] = pd.to_datetime(trans['date'])
trans['year'], trans['month'] = trans['date'].dt.year, trans['date'].dt.month

In [10]:
## 3. ABBREVIATED CLUB NAMES
adelaide = ['Adelaide']
brisbane = ['Brisbane Bears','Brisbane Lions']
carlton = ['Carlton']
collingwood = ['Collingwood']
essendon = ['Essendon']
fitzroy = ['Fitzroy']
footscray = ['Footscray']
fremantle = ['Fremantle']
geelong = ['Geelong']
gold_coast = ['Gold Coast']
gws = ['GW Sydney']
hawthorn = ['Hawthorn']
melbourne = ['Melbourne']
north_melbourne = ['North Melbourne','Kangaroos']
port_adelaide = ['Port Adelaide']
richmond = ['Richmond']
south_melbourne = ['South Melbourne']
st_kilda = ['St Kilda']
sydney = ['Sydney']
west_coast = ['West Coast']
western_bulldogs = ['Western Bulldogs', 'Footscray']
university = ['University']

In [11]:
def team_shorten(x):
    if x in adelaide:
        return "AD"
    elif x in brisbane:
        return "BR"
    elif x in carlton:
        return "CA"
    elif x in collingwood:
        return "CO"
    elif x in essendon:
        return "ES"
    elif x in fitzroy:
        return "FI"
    elif x in fremantle:
        return "FR"
    elif x in geelong:
        return "GE"
    elif x in gold_coast:
        return "GC"
    elif x in gws:
        return "GW"
    elif x in hawthorn:
        return "HA"
    elif x in melbourne:
        return "ME"
    elif x in north_melbourne:
        return "NM"
    elif x in port_adelaide:
        return "PA"
    elif x in richmond:
        return "RI"
    elif x in south_melbourne:
        return "SM"
    elif x in st_kilda:
        return "SK"
    elif x in sydney:
        return "SY"
    elif x in west_coast:
        return "WC"
    elif x in western_bulldogs:
        return "WB"
    elif x in university:
        return "UN"
    else: 
        return "NOT_FOUND"

In [12]:
trans["t1"] = trans["team1"].apply(team_shorten)
trans["t2"] = trans["team2"].apply(team_shorten)

In [13]:
trans['t1'].value_counts().sum()

15398

##  FIX TEAM NAMES ANOMOLIES

* Merge Footscray & Western Bulldogs as **Western Bulldogs**
* Merge North Melbourne with Kangaroos as **North Melbourne**
* Merge Brisbane Lions with Brisbane Bears as **Brisbane**
* Leave Sydney and **South Melbourne** as separate
* Leave **Fitzroy** as separate
* Modify GW Sydney to **GWS Giants**

In [14]:
# Function from Analytics Vidhya
def coding(col, codeDict):
  colCoded = pd.Series(col, copy=True)
  for key, value in codeDict.items():
    colCoded.replace(key, value, inplace=True)
  return colCoded

In [15]:
trans["team1"] = coding(trans["team1"], {'Footscray':'Western Bulldogs'})
trans["team2"] = coding(trans["team2"], {'Footscray':'Western Bulldogs'})
trans["team1"] = coding(trans["team1"], {'Kangaroos':'North Melbourne'})
trans["team2"] = coding(trans["team2"], {'Kangaroos':'North Melbourne'})
trans["team1"] = coding(trans["team1"], {'Brisbane Bears':'Brisbane','Brisbane Lions':'Brisbane'})
trans["team2"] = coding(trans["team2"], {'Brisbane Bears':'Brisbane','Brisbane Lions':'Brisbane'})
trans["team1"] = coding(trans["team1"], {'GW Sydney':'GWS Giants'})
trans["team2"] = coding(trans["team2"], {'GW Sydney':'GWS Giants'})

In [16]:
trans["team2"].value_counts()

Collingwood         1273
Carlton             1251
Essendon            1222
Geelong             1218
St Kilda            1190
Melbourne           1190
Richmond            1118
North Melbourne      986
Hawthorn             978
Western Bulldogs     975
Fitzroy              960
South Melbourne      788
Sydney               431
West Coast           376
Brisbane             359
Adelaide             325
Fremantle            273
Port Adelaide        254
Gold Coast            88
GWS Giants            80
University            63
Name: team2, dtype: int64

## ALL DRAWS (BOOLEAN)

In [17]:
def reg_draw(hvar):
    if hvar == 0:
        return 1
    else: return 0

In [18]:
trans["draw_rt"] = trans["hvar"].apply(reg_draw)

In [19]:
trans['draw_rt'].value_counts()

0    15240
1      158
Name: draw_rt, dtype: int64

### The 158 draws above, represent the VFL/AFL games that have concluded in a draw result

### The two ET (extra-time) draws below are statistically relevent, since they also ended regular full-time with both teams on the same score. However extra-time was applied to create a result, due to finals constraints.

In [20]:
def et_draw(seq):
    if seq == 10794:
        return 1
    elif seq == 13203:
        return 1
    else: return 0

In [21]:
trans["etime_draw"] = trans["seq"].apply(et_draw)
trans['etime_draw'].value_counts()

0    15396
1        2
Name: etime_draw, dtype: int64

In [22]:
### Add the two columns
trans['draws_all']  = trans['draw_rt'] + trans['etime_draw']
trans['draws_all'].value_counts()

0    15238
1      160
Name: draws_all, dtype: int64

In [23]:
# Check
trans.iloc[:,:][(trans.etime_draw == 1)]

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,...,state,hvar,aggr,year,month,t1,t2,draw_rt,etime_draw,draws_all
10794,10794,1994-09-10,QF,North Melbourne,15,24,114,Hawthorn,13,13,...,VIC,23,205,1994,9,NM,HA,0,1,1
13203,13203,2007-09-14,SF,Collingwood,13,15,93,West Coast,10,14,...,WA,19,167,2007,9,CO,WC,0,1,1


In [24]:
# 6. GAME CATEGORY
finals = ['EF','QF','SF','PF','GF']

In [25]:
def game_cat(x):
    if x in finals:
        return "final"
    else: 
        return "reg_season"

In [26]:
trans["type"] = trans["round"].apply(game_cat)

In [27]:
trans['type'].value_counts()

reg_season    14752
final           646
Name: type, dtype: int64

### So, there have been 646 finals up to the end of the 2017 season.

In [28]:
def grouping_2K(seq):
    n=500
    if seq <= 4*n:
        return 1
    elif seq <= 8*n:
        return 2
    elif seq <= 12*n:
        return 3
    elif seq <= 16*n:
        return 4
    elif seq <= 20*n:
        return 5
    elif seq <= 24*n:
        return 6
    elif seq <= 28*n:
        return 7
    else: return 8

In [29]:
trans["grp_2K"] = trans["seq"].apply(grouping_2K)

In [30]:
def grouping_1K(seq):
    n=1000
    if seq <= n:
        return 1
    elif seq <= 2*n:
        return 2
    elif seq <= 3*n:
        return 3
    elif seq <= 4*n:
        return 4
    elif seq <= 5*n:
        return 5
    elif seq <= 6*n:
        return 6
    elif seq <= 7*n:
        return 7
    elif seq <= 8*n:
        return 8
    elif seq <= 9*n:
        return 9
    elif seq <= 10*n:
        return 10
    elif seq <= 11*n:
        return 11
    elif seq <= 12*n:
        return 12
    elif seq <= 13*n:
        return 13
    elif seq <= 14*n:
        return 14
    elif seq <= 15*n:
        return 15
    else: return 16

In [31]:
trans["grp_1K"] = trans["seq"].apply(grouping_1K)

In [32]:
# Export the data frame for use in afl draws EDA
trans.to_csv('all_afl_wide.csv')

## ADD res1 and res2 columns as categorical 1 = win, 2 = loss, 3 = draw based on hvar (ignoring the two extra time finals)

In [33]:
def result_cat_h(hvar):
    if hvar > 0:
        return 1
    elif hvar < 0:
        return 2
    else: return 3

In [34]:
def result_cat_a(hvar):
    if hvar > 0:
        return 2
    elif hvar < 0:
        return 1
    else: return 3

In [35]:
trans['res1'] = trans['hvar'].apply(result_cat_h)
trans['res2'] = trans['hvar'].apply(result_cat_a)

In [36]:
trans.tail(9)

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,...,t1,t2,draw_rt,etime_draw,draws_all,type,grp_2K,grp_1K,res1,res2
15390,15390,2018-08-24,R23,Port Adelaide,13,11,89,Essendon,17,9,...,PA,ES,0,0,0,reg_season,8,16,2,1
15391,15391,2018-08-25,R23,Geelong,22,10,142,Gold Coast,5,10,...,GE,GC,0,0,0,reg_season,8,16,1,2
15392,15392,2018-08-25,R23,Richmond,15,8,98,Western Bulldogs,14,11,...,RI,WB,0,0,0,reg_season,8,16,1,2
15393,15393,2018-08-25,R23,Fremantle,9,13,67,Collingwood,11,10,...,FR,CO,0,0,0,reg_season,8,16,2,1
15394,15394,2018-08-25,R23,Carlton,8,13,61,Adelaide,26,9,...,CA,AD,0,0,0,reg_season,8,16,2,1
15395,15395,2018-08-25,R23,Sydney,10,14,74,Hawthorn,12,11,...,SY,HA,0,0,0,reg_season,8,16,2,1
15396,15396,2018-08-26,R23,Brisbane,11,6,72,West Coast,14,14,...,BR,WC,0,0,0,reg_season,8,16,2,1
15397,15397,2018-08-26,R23,Melbourne,15,12,102,GWS Giants,8,9,...,ME,GW,0,0,0,reg_season,8,16,1,2
15398,15398,2018-08-26,R23,St Kilda,14,10,94,North Melbourne,17,15,...,SK,NM,0,0,0,reg_season,8,16,2,1


## STAGE TWO
## 9. Reduce and re-order the data frame to prepare for for Long format

In [37]:
tracker = trans
tracker = tracker[['year', 'team1','pts_1','team2','pts_2','venue','t1','t2','hvar']]
tracker.tail()

,year,team1,pts_1,team2,pts_2,venue,t1,t2,hvar
15394,2018,Carlton,61,Adelaide,165,Docklands,CA,AD,-104
15395,2018,Sydney,74,Hawthorn,83,S.C.G.,SY,HA,-9
15396,2018,Brisbane,72,West Coast,98,Gabba,BR,WC,-26
15397,2018,Melbourne,102,GWS Giants,57,M.C.G.,ME,GW,45
15398,2018,St Kilda,94,North Melbourne,117,Docklands,SK,NM,-23


### Create two new columns with values 1 and 0 for the two subsets

In [38]:
tracker['subset_1'] = 1
tracker['subset_2'] = 0

/Applications/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Applications/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [39]:
tracker.tail(9)

,year,team1,pts_1,team2,pts_2,venue,t1,t2,hvar,subset_1,subset_2
15390,2018,Port Adelaide,89,Essendon,111,Adelaide Oval,PA,ES,-22,1,0
15391,2018,Geelong,142,Gold Coast,40,Kardinia Park,GE,GC,102,1,0
15392,2018,Richmond,98,Western Bulldogs,95,M.C.G.,RI,WB,3,1,0
15393,2018,Fremantle,67,Collingwood,76,Perth Stadium,FR,CO,-9,1,0
15394,2018,Carlton,61,Adelaide,165,Docklands,CA,AD,-104,1,0
15395,2018,Sydney,74,Hawthorn,83,S.C.G.,SY,HA,-9,1,0
15396,2018,Brisbane,72,West Coast,98,Gabba,BR,WC,-26,1,0
15397,2018,Melbourne,102,GWS Giants,57,M.C.G.,ME,GW,45,1,0
15398,2018,St Kilda,94,North Melbourne,117,Docklands,SK,NM,-23,1,0


In [40]:
droplist_1 = ['subset_2']
droplist_2 = ['subset_1']

In [41]:
left_team = tracker.drop(droplist_1, axis = 1)
right_team = tracker.drop(droplist_2, axis = 1)

In [42]:
left_team.shape

(15398, 10)

In [43]:
right_team.shape

(15398, 10)

# Rename columns in each dframe 

In [44]:
left_team.columns = ['year','team','tm_pts','opp_team','op_pts','venue','tm','op','hvar','lr']
right_team.columns = ['year','opp_team','op_pts','team','tm_pts','venue','op','tm','hvar','lr']

### Now re-order the second dframe to match columns

### Note double brackets to re-order

In [45]:
right_team = right_team[['year', 'team','tm_pts','opp_team','op_pts','venue','tm','op','hvar','lr']]
right_team.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15394,2018,Adelaide,165,Carlton,61,Docklands,AD,CA,-104,0
15395,2018,Hawthorn,83,Sydney,74,S.C.G.,HA,SY,-9,0
15396,2018,West Coast,98,Brisbane,72,Gabba,WC,BR,-26,0
15397,2018,GWS Giants,57,Melbourne,102,M.C.G.,GW,ME,45,0
15398,2018,North Melbourne,117,St Kilda,94,Docklands,NM,SK,-23,0


In [46]:
right_team.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15394,2018,Adelaide,165,Carlton,61,Docklands,AD,CA,-104,0
15395,2018,Hawthorn,83,Sydney,74,S.C.G.,HA,SY,-9,0
15396,2018,West Coast,98,Brisbane,72,Gabba,WC,BR,-26,0
15397,2018,GWS Giants,57,Melbourne,102,M.C.G.,GW,ME,45,0
15398,2018,North Melbourne,117,St Kilda,94,Docklands,NM,SK,-23,0


In [47]:
left_team.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15394,2018,Carlton,61,Adelaide,165,Docklands,CA,AD,-104,1
15395,2018,Sydney,74,Hawthorn,83,S.C.G.,SY,HA,-9,1
15396,2018,Brisbane,72,West Coast,98,Gabba,BR,WC,-26,1
15397,2018,Melbourne,102,GWS Giants,57,M.C.G.,ME,GW,45,1
15398,2018,St Kilda,94,North Melbourne,117,Docklands,SK,NM,-23,1


## CONCATENATION

### Now very easily, can concatenate the two dframes, as in si_week11

In [48]:
merge_ft = pd.concat([left_team, right_team], axis=0)
merge_ft.head()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
1,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1
2,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1
3,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1
4,1897,South Melbourne,27,Melbourne,44,Lake Oval,SM,ME,-17,1
5,1897,South Melbourne,40,Carlton,36,Lake Oval,SM,CA,4,1


#### The new long data frame consisting of 30,000 plus rows:

In [49]:
merge_ft.shape

(30796, 10)

# The following is to keep each game together (two rows per game)

In [50]:
merge_ft.sort_index(inplace=True)
merge_ft.head(10)

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
1,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1
1,1897,Carlton,16,Fitzroy,49,Brunswick St,CA,FI,33,0
2,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1
2,1897,St Kilda,16,Collingwood,41,Victoria Park,SK,CO,25,0
3,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1
3,1897,Essendon,47,Geelong,24,Corio Oval,ES,GE,-23,0
4,1897,South Melbourne,27,Melbourne,44,Lake Oval,SM,ME,-17,1
4,1897,Melbourne,44,South Melbourne,27,Lake Oval,ME,SM,-17,0
5,1897,South Melbourne,40,Carlton,36,Lake Oval,SM,CA,4,1
5,1897,Carlton,36,South Melbourne,40,Lake Oval,CA,SM,4,0


In [51]:
merge_ft.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15396,2018,Brisbane,72,West Coast,98,Gabba,BR,WC,-26,1
15397,2018,GWS Giants,57,Melbourne,102,M.C.G.,GW,ME,45,0
15397,2018,Melbourne,102,GWS Giants,57,M.C.G.,ME,GW,45,1
15398,2018,St Kilda,94,North Melbourne,117,Docklands,SK,NM,-23,1
15398,2018,North Melbourne,117,St Kilda,94,Docklands,NM,SK,-23,0


## There are actually 30,000 plus rows, indexing isn't simple sequential

## Create team result column

In [52]:
def neg_hvar(hvar):
        return -1*hvar

In [53]:
merge_ft["neg_dummy"] = merge_ft['hvar'].apply(neg_hvar)

In [54]:
merge_ft.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,neg_dummy
15396,2018,Brisbane,72,West Coast,98,Gabba,BR,WC,-26,1,26
15397,2018,GWS Giants,57,Melbourne,102,M.C.G.,GW,ME,45,0,-45
15397,2018,Melbourne,102,GWS Giants,57,M.C.G.,ME,GW,45,1,-45
15398,2018,St Kilda,94,North Melbourne,117,Docklands,SK,NM,-23,1,23
15398,2018,North Melbourne,117,St Kilda,94,Docklands,NM,SK,-23,0,23


In [55]:
def team_var(hvar):
    if merge_ft['lr'] == 1:
        return hvar
    else: return neg_dummy

In [56]:
def new_lr(lr):
    if lr == 0:
        return -1
    else: return 1

In [57]:
merge_ft["new_lr"] = merge_ft['lr'].apply(new_lr)
merge_ft["tm_var"] = merge_ft['new_lr']*merge_ft['hvar']

In [58]:
merge_ft.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,neg_dummy,new_lr,tm_var
15396,2018,Brisbane,72,West Coast,98,Gabba,BR,WC,-26,1,26,1,-26
15397,2018,GWS Giants,57,Melbourne,102,M.C.G.,GW,ME,45,0,-45,-1,-45
15397,2018,Melbourne,102,GWS Giants,57,M.C.G.,ME,GW,45,1,-45,1,45
15398,2018,St Kilda,94,North Melbourne,117,Docklands,SK,NM,-23,1,23,1,-23
15398,2018,North Melbourne,117,St Kilda,94,Docklands,NM,SK,-23,0,23,-1,23


### Can now remove the columns new_lr and neg_dummy, which were used to create tm_var

In [59]:
merge_ft = merge_ft.drop(['neg_dummy', 'new_lr'], axis=1)

In [60]:
merge_ft.head()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var
1,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1,33
1,1897,Carlton,16,Fitzroy,49,Brunswick St,CA,FI,33,0,-33
2,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1,25
2,1897,St Kilda,16,Collingwood,41,Victoria Park,SK,CO,25,0,-25
3,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1,-23


### Note that the column 'lr' denotes left right, so there are only two different values (1 denotes left)

In [61]:
merge_ft['lr'].value_counts()

1    15398
0    15398
Name: lr, dtype: int64

## New function for win/loss/draw (W/L/D)


In [62]:
def outcome(tm_var):
    if tm_var >= 1:
        return "W"
    elif tm_var <= -1:
        return "L"
    elif tm_var == 0:
        return "D"
    else: return "Unknown"

In [63]:
merge_ft["outcome"] = merge_ft['tm_var'].apply(outcome)

In [64]:
merge_ft.head()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome
1,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1,33,W
1,1897,Carlton,16,Fitzroy,49,Brunswick St,CA,FI,33,0,-33,L
2,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1,25,W
2,1897,St Kilda,16,Collingwood,41,Victoria Park,SK,CO,25,0,-25,L
3,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1,-23,L


In [66]:
merge_ft.tail(9)

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome
15394,2018,Carlton,61,Adelaide,165,Docklands,CA,AD,-104,1,-104,L
15395,2018,Hawthorn,83,Sydney,74,S.C.G.,HA,SY,-9,0,9,W
15395,2018,Sydney,74,Hawthorn,83,S.C.G.,SY,HA,-9,1,-9,L
15396,2018,West Coast,98,Brisbane,72,Gabba,WC,BR,-26,0,26,W
15396,2018,Brisbane,72,West Coast,98,Gabba,BR,WC,-26,1,-26,L
15397,2018,GWS Giants,57,Melbourne,102,M.C.G.,GW,ME,45,0,-45,L
15397,2018,Melbourne,102,GWS Giants,57,M.C.G.,ME,GW,45,1,45,W
15398,2018,St Kilda,94,North Melbourne,117,Docklands,SK,NM,-23,1,-23,L
15398,2018,North Melbourne,117,St Kilda,94,Docklands,NM,SK,-23,0,23,W


## Example filtering view

# Hawks last 10 games againt Richmond

In [67]:
merge_ft.iloc[:,:][(merge_ft.tm == 'HA') & (merge_ft.op == 'RI')].tail(10)

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome
13639,2010,Hawthorn,89,Richmond,86,M.C.G.,HA,RI,-3,0,3,W
13783,2011,Hawthorn,115,Richmond,52,M.C.G.,HA,RI,-63,0,63,W
14033,2012,Hawthorn,75,Richmond,137,M.C.G.,HA,RI,62,0,-62,L
14322,2013,Hawthorn,66,Richmond,107,M.C.G.,HA,RI,-41,1,-41,L
14426,2014,Hawthorn,118,Richmond,52,M.C.G.,HA,RI,-66,0,66,W
14724,2015,Hawthorn,53,Richmond,71,M.C.G.,HA,RI,-18,1,-18,L
14841,2016,Hawthorn,136,Richmond,90,M.C.G.,HA,RI,-46,0,46,W
14938,2016,Hawthorn,114,Richmond,44,M.C.G.,HA,RI,70,1,70,W
15163,2017,Hawthorn,64,Richmond,93,M.C.G.,HA,RI,29,0,-29,L
15225,2018,Hawthorn,89,Richmond,102,M.C.G.,HA,RI,13,0,-13,L


## Demon's last 10 games against Geelong

In [68]:
merge_ft.iloc[:,:][(merge_ft.tm == 'ME') & (merge_ft.op == 'GE')].tail(10)

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome
13651,2010,Melbourne,67,Geelong,121,Kardinia Park,ME,GE,54,0,-54,L
13906,2011,Melbourne,47,Geelong,233,Kardinia Park,ME,GE,186,0,-186,L
14007,2012,Melbourne,76,Geelong,119,Kardinia Park,ME,GE,43,0,-43,L
14295,2013,Melbourne,30,Geelong,98,Kardinia Park,ME,GE,68,0,-68,L
14510,2014,Melbourne,36,Geelong,102,M.C.G.,ME,GE,-66,1,-66,L
14682,2015,Melbourne,113,Geelong,89,Kardinia Park,ME,GE,-24,0,24,W
14977,2016,Melbourne,44,Geelong,155,Kardinia Park,ME,GE,111,0,-111,L
15015,2017,Melbourne,97,Geelong,126,Docklands,ME,GE,29,0,-29,L
15208,2018,Melbourne,94,Geelong,97,M.C.G.,ME,GE,-3,1,-3,L
15350,2018,Melbourne,98,Geelong,100,Kardinia Park,ME,GE,2,0,-2,L


## Swans's last 5 games against GWS

In [70]:
merge_ft.iloc[:,:][(merge_ft.tm == 'SY') & (merge_ft.op == 'GW') ].tail(5)

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome
14987,2016,Sydney,55,GWS Giants,91,Stadium Australia,SY,GW,36,0,-36,L
15033,2017,Sydney,63,GWS Giants,105,S.C.G.,SY,GW,-42,1,-42,L
15133,2017,Sydney,96,GWS Giants,83,Sydney Showground,SY,GW,-13,0,13,W
15224,2018,Sydney,103,GWS Giants,87,S.C.G.,SY,GW,16,1,16,W
15384,2018,Sydney,80,GWS Giants,60,Sydney Showground,SY,GW,-20,0,20,W


## Pies last 8

In [71]:
merge_ft.iloc[:,:][(merge_ft.tm == 'CO') & (merge_ft.year == 2018)].tail(8)

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome
15334,2018,Collingwood,78,Essendon,62,M.C.G.,CO,ES,-16,0,16,W
15342,2018,Collingwood,67,West Coast,102,M.C.G.,CO,WC,-35,1,-35,L
15346,2018,Collingwood,130,North Melbourne,64,M.C.G.,CO,NM,66,1,66,W
15355,2018,Collingwood,77,Richmond,105,M.C.G.,CO,RI,28,0,-28,L
15367,2018,Collingwood,71,Sydney,73,S.C.G.,CO,SY,2,0,-2,L
15377,2018,Collingwood,104,Brisbane,73,Docklands,CO,BR,31,1,31,W
15382,2018,Collingwood,115,Port Adelaide,64,M.C.G.,CO,PA,51,1,51,W
15393,2018,Collingwood,76,Fremantle,67,Perth Stadium,CO,FR,-9,0,9,W


## Ballarat games - check venue is shown correctly

In [72]:
merge_ft.iloc[:,:][(merge_ft.tm == 'WB') & (merge_ft.venue != 'Docklands') & (merge_ft.year == 2017)].tail(10)

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome
15017,2017,Western Bulldogs,73,Fremantle,89,Subiaco,WB,FR,16,0,-16,L
15039,2017,Western Bulldogs,73,GWS Giants,75,Manuka Oval,WB,GW,2,0,-2,L
15057,2017,Western Bulldogs,61,West Coast,69,Subiaco,WB,WC,8,0,-8,L
15066,2017,Western Bulldogs,81,Geelong,104,Kardinia Park,WB,GE,23,0,-23,L
15089,2017,Western Bulldogs,42,Sydney,88,S.C.G.,WB,SY,46,0,-46,L
15120,2017,Western Bulldogs,45,Adelaide,104,Adelaide Oval,WB,AD,59,0,-59,L
15136,2017,Western Bulldogs,82,Carlton,62,M.C.G.,WB,CA,-20,0,20,W
15141,2017,Western Bulldogs,110,Gold Coast,56,Cazaly's Stadium,WB,GC,54,1,54,W
15159,2017,Western Bulldogs,103,Brisbane,89,Gabba,WB,BR,-14,0,14,W
15175,2017,Western Bulldogs,79,Port Adelaide,96,Eureka Stadium,WB,PA,-17,1,-17,L
